In [1]:
%matplotlib inline
import numpy as np
import pulp
import pandas as pd
from __future__ import print_function
import pulp_lineup_optimization as lopt
import datetime
import urllib2
import re
import scipy.special as scsp
import time
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,8)

In [3]:
### CONSTANTS ###

TEAM_ABBREVS = {
    'ATL': 'Atlanta Hawks',
    'BKN': 'Brooklyn Nets',
    'BOS': 'Boston Celtics',
    'CHA': 'Charlotte Hornets',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Caveliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'Los Angeles Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIN': 'Minnesota Timberwolves',
    'MIL': 'Milwaukee Bucks',
    'NO': 'New Orleans Pelicans',
    'NY': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHO': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SA': 'San Antonio Spurs',
    'SAC': 'Sacramento Kings',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards'
}

NAME2ABBREV = {
    'Atlanta': 'ATL',
    'Boston': 'BOS',
    'Brooklyn': 'BKN',
    'Charlotte': 'CHA',
    'Chicago': 'CHI',
    'Cleveland': 'CLE',
    'Dallas': 'DAL',
    'Denver': 'DEN',
    'Detroit': 'DET',
    'Golden State': 'GS',
    'Houston': 'HOU',
    'Indiana': 'IND',
    'L.A. Clippers': 'LAC',
    'L.A. Lakers': 'LAL',
    'Memphis': 'MEM',
    'Miami': 'MIA',
    'Milwaukee': 'MIL',
    'Minnesota': 'MIN',
    'New Orleans': 'NO',
    'New York': 'NY',
    'Oklahoma City': 'OKC',
    'Orlando': 'ORL',
    'Philadelphia': 'PHI',
    'Phoenix': 'PHO',
    'Portland': 'POR',
    'Sacramento': 'SAC',
    'San Antonio': 'SA',
    'Toronto': 'TOR',
    'Utah': 'UTA',
    'Washington': 'WAS'
}

## Get Depth Charts

In [3]:
def get_depth_charts():
    tabs = pd.read_html('http://www.rotoworld.com/teams/depth-charts/nba.aspx')

    depth_charts = {}
    for k, team in enumerate(sorted(TEAM_ABBREVS.keys())):
        tab = tabs[k+1].copy()
        N = len(tab[0])
        dic = {'Name': [], 'Pos': [], 'Rank': [], 'Out': [], 'Notes': []}
        for n in xrange(N):
            tmp_pos = tab[0].loc[n]
            if tmp_pos in ('PG','SG','SF','PF','C'):
                pos = tmp_pos
                rk = 1
                dic[pos] = {}
            val = tab[1].loc[n][3:]
            search = re.search('.{3,35} Sidelined', val)
            if search:
                name = search.group(0)[:-10]
                search = re.search('Sidelined.{0,1000}', val)
                notes = search.group(0)[9:]
            else:
                search = re.search('.{3,35} Suspended', val)
                if search:
                    name = search.group(0)[:-10]
                    search = re.search('Sidelined.{0,1000}', val)
                    notes = search.group(0)[9:]
                else:
                    name = val
                    notes = ''
            dic['Name'].append(name)
            dic['Notes'].append(notes)
            dic['Out'].append(notes!='')
            dic['Pos'].append(pos)
            dic['Rank'].append(rk)
            rk += 1


        index = pd.MultiIndex.from_tuples(zip(dic['Pos'],dic['Rank']), names=['Pos','Rank'])
        depth_charts[team] = pd.DataFrame(dic, columns=['Name','Out','Notes'], index=index)
    return depth_charts

def adjust_depth_chart(dc):
    '''
    Adjusts players rank based on any players Sidelined above them.
    '''
    pass
depth_charts = get_depth_charts()

ImportError: html5lib not found, please install it

# Get Historical Salaries

In [59]:
dfs = []

for site in ['fd','dk']:
    url = 'http://rotoguru1.com/cgi-bin/hyday.pl?mon=%%(month)02d&day=%%(day)02d&year=%%(year)d&game=%(site)s' % {'site':site}
    rows = []

    game_date = datetime.date(2014,10,28)
    while game_date <= datetime.date(2015,4,15):
        date = {'month': game_date.month,
                'day':   game_date.day,
                'year':  game_date.year}
        page = urllib2.urlopen(url % date)

        for line in page:
            if any([pos in line for pos in ('<td>PG</td>','<td>SG</td>','<td>SF</td>','<td>PF</td>','<td>C</td>')]):
                row = line.split('</td><td')
                pos = row[0][-2:]
                if pos[0] == '>':
                    pos = pos[1:]
                pos = pos.strip()
                name = re.search('>.{4,30}<', row[1]).group(0)[1:-1].strip()
                starter = int('</a>^' in row[1])
                fp = float(re.search('>.{1,5}', row[2]).group(0)[1:])
                try:
                    sal = int(re.search('>.{1,7}', row[3]).group(0)[1:].replace(',','').replace('$',''))
                except ValueError:
                    sal = None
                team = re.search('>...', row[4]).group(0)[1:].upper().strip()
                c, opp = re.search('>.....', row[5]).group(0)[1:].split(' ')
                h_a = 'H' if c=='v' else 'A'
                opp = opp.upper().strip()
                mins = float(re.search('>.{1,6}', row[6]).group(0)[1:])
                gl = re.search('> &nbsp; .{0,100}</td></tr>', row[7]).group(0)[9:-10].strip()
                rows.append([game_date, name, pos, starter, fp, sal, team, h_a, opp, mins, gl])

        game_date += datetime.timedelta(days=1)

    dfs.append(pd.DataFrame(rows, columns=('Game_Date','Name','Pos','Starter','FP','Salary','Team','H_A','Opp','Min','Stats')))
fd = dfs[0]
dk = dfs[1]

In [ ]:
### Write Results to CSV ###
fd.write_csv('../nba/data/fd/rotoguru.csv')
dk.write_csv('../nba/data/dk/rotoguru.csv')

## Lineup Optimization with My Projections

#### Set Up Problem

In [8]:
PROJ = pd.read_csv('Projections/nba_projections_11_30_2015.csv')

FDorDK = True
FDorDK = False

PROJ = PROJ[~np.isnan(PROJ.Played)]
games = np.unique(PROJ.Game).tolist()
print(games)
nlineups = 1
player_limit = 2
risk_aversion = 0
Gamma = 4
DvPrnkLimit = 20

if not FDorDK:
    ## DraftKings
    # {'PG': 1, 'SG': 1, 'SF': 1, 'PF': 1, 'C': 1, 'PG/SG': 1, 'SF/PF': 1, 'PG/SG/SF/PF/C': 1}
    nslots = 8
    salary_cap = 50000
    tab = PROJ.copy()
    tab['Proj'] = tab.DK - risk_aversion * tab.DK_std
#     tab['Proj'] = tab['Actual_DK']
    tab['Salary'] = tab.dk_Salary
    tab['Pos'] = tab.dk_Pos
    tab['Actual'] = tab.Actual_DK
    tab.Pos = tab.Pos.fillna('')
    tab = tab[tab.Pos != '']
else:
    ## FanDuel
    # {'PG': 2, 'SG': 2, 'SF': 2, 'PF': 2, 'C': 1}
    nslots = 9
    salary_cap = 60000
    tab = PROJ.copy()
    tab['Proj'] = tab.FD - risk_aversion * tab.FD_std
#     tab['Proj'] = tab['Actual_FD']
    tab['Salary'] = tab.fd_Salary
    tab['Pos'] = tab.fd_Pos
    tab['Actual'] = tab.Actual_FD

# Use only players in the selected games
teams = [x for y in games for x in y.split('@')]
games_dic = [x.split('@') for x in games]
games_dic.extend([x.split('@')[::-1] for x in games])
games_dic = dict(games_dic)
tab = tab[np.array([t in teams for t in tab.Team], dtype=bool)]

# Only consider worth while players... (depends on number of games in pool)
tab.Proj = tab.Proj.fillna(0)
tab = tab[tab.Proj>10]

# Make sure salary is provided
tab = tab[np.logical_not(np.isnan(tab.Played))]

# Remove sidelined players
tab = tab[(tab.Status!='O')&(tab.Status!='IR')&(tab.Status!='NA')]

# This is for ordering players in the table by position
pos2num = {'PG': 1, 'SG': 2, 'SF': 3, 'PF': 4, 'C': 5}
tab['PosNum'] = [pos2num[x] for x in tab.Pos]
tab.Status = tab.Status.fillna('')

# For robust option
tab['LB'] = tab.Proj*.8
tab['UB'] = tab.Proj

# Set Descriptive Indexing
tab['TAG'] = ['%s %s (%s, %s)' % (f,l,p,t) for f,l,p,t in zip(tab['First Name'], tab['Last Name'], tab.Pos, tab.Team)]

# Defense Quality Filter
dvp = pd.read_html('https://rotogrinders.com/pages/nba-defense-vs-position-stats-all-positions-season-176473')[1]
dvp = dvp.rename(columns=dict(zip(dvp.keys(),[x.strip() for x in dvp.keys()])))
dvp.Team = [NAME2ABBREV[t] for t in dvp.Team]
dvp = dvp.set_index('Team')
exceptions = ['Greg Monroe (C, MIL)']
I = [n in exceptions or dvp['%sRk' % p].loc[games_dic[t]] >= DvPrnkLimit for t, p, n in zip(tab.Team,tab.Pos,tab.TAG)]
tab = tab[I]

tab = tab.set_index('TAG')

# Lineup Appearance Limit
UNLIMITED_APPEARANCES = [
]
nplayers = len(tab.Proj)
try:
    tab['Own Limit'] = tab['Own % Limit']*nlineups
except KeyError:
    tab['Own Limit'] = nplayers*[player_limit]
    if len(UNLIMITED_APPEARANCES) > 0:
        tab.set_value(UNLIMITED_APPEARANCES, 'Own Limit', len(UNLIMITED_APPEARANCES)*[nlineups])

## Create Optimizer
# lo = lopt.RobustMultiLineupOptimizer(tab, nslots, Gamma, nlineups, salary_cap)
lo = lopt.MultiLineupOptimizer(tab, nslots, nlineups, salary_cap)

# These constraints are specified on the websites
if FDorDK:
    for j in xrange(nlineups):
        # Roster Constraint
        lo.addPositionConstraint('PG', 'ge', 1, j)
        lo.addPositionConstraint('PG', 'le', 3, j)
        lo.addPositionConstraint('SG', 'ge', 1, j)
        lo.addPositionConstraint('SG', 'le', 3, j)
        lo.addPositionConstraint('SF', 'ge', 1, j)
        lo.addPositionConstraint('SF', 'le', 3, j)
        lo.addPositionConstraint('PF', 'ge', 1, j)
        lo.addPositionConstraint('PF', 'le', 3, j)
        lo.addPositionConstraint('C', 'ge', 1, j)
        lo.addPositionConstraint('C', 'le', 2, j)
        lo.addPositionConstraint('PG/SG', 'ge', 3, j)
        lo.addPositionConstraint('SF/PF', 'ge', 3, j)
        lo.addPositionConstraint('PG/SG/SF/PF/C', 'eq', 8, j)
        # At least players from 2 teams
        for team in np.unique(tab.Team):
            lo.addTeamLimitConstraint(team, 'le', 7, j)
        # At least players from 2 games
        for game in games:
            lo.addTeamLimitConstraint(game.replace('@','/'), 'le', 7, j)
else:
    for j in xrange(nlineups):
        # Roster Constraint
        lo.addPositionConstraint('PG', 'eq', 2, j)
        lo.addPositionConstraint('SG', 'eq', 2, j)
        lo.addPositionConstraint('SF', 'eq', 2, j)
        lo.addPositionConstraint('PF', 'eq', 2, j)
        lo.addPositionConstraint('C', 'eq', 1, j)
        # At least players from 3 teams
        for k, team in enumerate(np.unique(tab.Team)):
            for team2 in np.unique(tab.Team)[k+1:]:
                lo.addTeamLimitConstraint('%s/%s' % (team,team2), 'le', 8, j)
            # Max 4 players from a single team
            lo.addTeamLimitConstraint(team, 'le', 4, j)

# Set Player Appearance Limits
for player in tab.index:
    if tab['Own Limit'].loc[player] < nlineups:
        lo.addPlayerConstraint(player, 'le', tab['Own Limit'].loc[player], -1)


['BOS@MIA', 'DAL@SAC', 'DEN@MIL', 'GS@UTA', 'HOU@DET', 'OKC@ATL', 'POR@LAC', 'SA@CHI']


In [18]:
## Random Constraints
# lo.addPlayerConstraint('', 'eq', 0, -1)
# lo.addPlayerConstraint('Ty Lawson (PG, HOU)', 'eq', 0, -1)
# lo.addPlayerConstraint('Chris Kaman (C, POR)', 'eq', 0, -1)
# lo.addPlayerConstraint('T.J. McConnell (PG, PHI)', 'eq', 0, -1)
# lo.addPlayerConstraint('Goran Dragic (PG, MIA)', 'eq', 0, -1)
# lo.addPlayerConstraint('Khris Middleton (SG, MIL)', 'eq', 0, -1)
# lo.addPlayerConstraint('Robin Lopez (C, NY)', 'eq', 0, -1)
# lo.addPlayerConstraint('Jarrett Jack (PG, BKN)', 'eq', 0, -1)
# lo.addPlayerConstraint('Rajon Rondo (PG, SAC)', 'eq', 0, -1)
# lo.addPlayerConstraint('Kyle Lowry (PG, TOR)', 'eq', 0, -1)
# lo.addPlayerConstraint('Danny Green (SG, SA)', 'eq', 0, -1)
# lo.addPlayerConstraint('Jerryd Bayless (PG, MIL)', 'eq', 0, -1)
# lo.addPlayerConstraint('Khris Middleton (SF, MIL)', 'eq', 0, -1)
# lo.addPlayerConstraint('Evan Fournier (SG, ORL)', 'eq', 0, -1)
# lo.addPlayerConstraint('Luke Babbitt (SF, NO)', 'eq', 0, -1)
# lo.addPlayerConstraint('Courtney Lee (SG, MEM)', 'eq', 0, -1)
lo.addPlayerConstraint('Luol Deng (SF, MIA)', 'eq', 0, -1)

# lo.addPlayerConstraint('', 'eq', 1, -1)
# lo.addPlayerConstraint('LaMarcus Aldridge (PF, SA)', 'eq', 1, -1)
# lo.addPlayerConstraint('Chris Paul (PG, LAC)', 'eq', 1, -1)
# lo.addPlayerConstraint('Kentavious Caldwell-Pope (SG, DET)', 'eq', 1, -1)
# lo.addPlayerConstraint('Marc Gasol (C, MEM)', 'eq', 1, -1)
# lo.addPlayerConstraint('Brandon Knight (PG, PHO)', 'eq', 1, -1)
# lo.addPlayerConstraint('James Harden (SG, HOU)', 'eq', 1, -1)

# lo.addPlayerConstraint('DeMarcus Cousins (PF, SAC)/Russell Westbrook (PG, OKC)', 'eq', 1, -1)

# lo.addTeamLimitConstraint('MIL', 'le', 3)

## Solve
print('Solving...')
%time status, lus = lo.solve()
print('Status: %s' % status)

lu_proj = []
lu_sal = []
for lu in lus:
    lu_proj.append(np.sum(lu.Proj))
    lu_sal.append(np.sum(lu.Salary))

xx = zip(lu_proj, lu_sal, lus)
xx = sorted(xx, key=lambda x: x[0], reverse=True)
for p, s, lu in xx:
    print()
    print('Projected Score: %.2f' % p)
    print('Total Salary: %d' % s)
    lu['PValue'] = lu.Proj / lu.Salary * 1000
    if FDorDK:
        lu['Actual'] = tab['Actual_DK'].loc[lu.index]
        lu['Sharpe'] = (lu.DK - 4./1000*lu.Salary)/lu.DK_std
    else:
        lu['Actual'] = tab['Actual_FD'].loc[lu.index]
        lu['Sharpe'] = (lu.FD - 4./1000*lu.Salary)/lu.FD_std

    lu['Value'] = lu.Actual / np.array(lu.Salary, dtype=float) * 1000
    print('Actual Score: %.2f' % np.sum(lu.Actual))
# lu[['Opp','Salary','Proj','PValue','Actual','Value','MIN','Actual_Mins']]
lu[['Opp','Salary','Proj','Sharpe','Status','MIN','GameTime']]

Solving...
CPU times: user 16 ms, sys: 8 ms, total: 24 ms
Wall time: 201 ms
Status: Optimal

Projected Score: 272.76
Total Salary: 50000
Actual Score: 0.00


,Opp,Salary,Proj,Sharpe,Status,MIN,GameTime
TAG,,,,,,,
"Deron Williams (PG, DAL)",SAC,6100,31.740516,0.430642,,34.131025,2015-11-30 22:00:00
"Raymond Felton (PG, DAL)",SAC,3200,19.078036,0.579484,,25.856837,2015-11-30 22:00:00
"Khris Middleton (SG, MIL)",DEN,5700,28.773950,0.315166,,36.470959,2015-11-30 20:00:00
"O.J. Mayo (SG, MIL)",DEN,4000,28.425220,0.765005,,35.428931,2015-11-30 20:00:00
"Kawhi Leonard (SF, SA)",CHI,8600,45.099320,0.495500,,36.855951,2015-11-30 20:00:00
"Marcus Morris (SF, DET)",HOU,5700,29.713059,0.386063,,37.299121,2015-11-30 19:30:00
"Dirk Nowitzki (PF, DAL)",SAC,6400,32.629526,0.377965,,31.545341,2015-11-30 22:00:00
"Ersan Ilyasova (PF, DET)",HOU,4300,21.748155,0.309279,,26.210193,2015-11-30 19:30:00
"Tim Duncan (C, SA)",CHI,6000,35.553735,0.846854,,29.484761,2015-11-30 20:00:00


In [19]:
if not FDorDK:
    lu.to_csv('DK_LINEUP.csv')
else:
    lu.to_csv('FD_LINEUP.csv')

In [19]:
pd.read_csv('FD_LINEUP1.csv')[['TAG','GameTime']]

,TAG,GameTime
0,"Chris Paul (PG, LAC)",2015-11-27 22:30:00
1,"Brandon Knight (PG, PHO)",2015-11-27 21:30:00
2,"Kentavious Caldwell-Pope (SG, DET)",2015-11-27 20:00:00
3,"Danny Green (SG, SA)",2015-11-27 21:00:00
4,"Khris Middleton (SF, MIL)",2015-11-27 19:00:00
5,"Justise Winslow (SF, MIA)",2015-11-27 19:30:00
6,"Blake Griffin (PF, LAC)",2015-11-27 22:30:00
7,"LaMarcus Aldridge (PF, SA)",2015-11-27 21:00:00
8,"Marc Gasol (C, MEM)",2015-11-27 20:00:00


# Read File for Lineup and Get Score

In [5]:
FDorDK = True
FDorDK = False

if FDorDK:
    fname = 'FD_LINEUP1.csv'
    pos = 'fd_Pos'
    site = 'FD'
    sal = 'fd_Salary'
else:
    fname = 'DK_LINEUP1.csv'
    pos = 'dk_Pos'
    site = 'DK'
    sal = 'dk_Salary'

cols = ['Opp',sal,site,'PValue','Actual_%s'%site,'Value','MIN','Actual_Mins']

TABLE = PROJ.copy()
TABLE['TAG'] = ['%s %s (%s, %s)' % (f,l,p,t) for f,l,p,t in zip(TABLE['First Name'], TABLE['Last Name'], TABLE[pos], TABLE.Team)]
LU = TABLE.set_index('TAG').loc[pd.read_csv(fname).TAG]
print(site)
print('Salary:', LU.fd_Salary.sum())
print('Projected:', LU.FD.sum())
print('Actual:', LU.Actual_FD.sum())
LU['PValue'] = LU[site] / LU[sal] * 1000
LU['Value'] = LU['Actual_%s'%site] / LU[sal] * 1000
LU[cols]

DK
Salary: 52800.0
Projected: 239.452336224
Actual: 185.3


,Opp,dk_Salary,DK,PValue,Actual_DK,Value,MIN,Actual_Mins
TAG,,,,,,,,
"Chris Paul (PG, LAC)",NO,8400,40.663851,4.840935,39.75,4.732143,33.066367,29
"Brandon Knight (PG, PHO)",GS,8200,39.242585,4.785681,32.25,3.932927,36.392376,35
"Kentavious Caldwell-Pope (SG, DET)",OKC,4800,24.836600,5.174292,23.50,4.895833,38.375327,40
"Justise Winslow (SF, MIA)",NY,4000,18.737627,4.684407,21.75,5.437500,30.848984,35
"Tony Allen (SF, MEM)",ATL,3500,19.506281,5.573223,15.00,4.285714,25.087629,20
"LaMarcus Aldridge (PF, SA)",DEN,6800,36.826120,5.415606,16.00,2.352941,33.081334,18
"Marc Gasol (C, MEM)",ATL,7200,39.045273,5.422955,25.00,3.472222,38.154103,37
"Greg Monroe (C, MIL)",ORL,7100,37.790414,5.322593,18.50,2.605634,34.347221,19


## Team Depth Charts

In [29]:
depth_charts = get_depth_charts()
for team in sorted(TEAM_ABBREVS.keys()):
    try:
        print(TEAM_ABBREVS[team])
        print(depth_charts[team][['Name','Out']])
    except KeyError:
        print(team)
        print(TEAM_ABBREVS['GSW'])
        print(depth_charts['GSW'][['Name','Out']])

Atlanta Hawks
                      Name    Out
Pos Rank                         
PG  1          Jeff Teague  False
    2      Dennis Schroder  False
    3         Shelvin Mack  False
    4      Lamar Patterson  False
SG  1          Kyle Korver  False
    2     Tim Hardaway Jr.  False
    3       Justin Holiday  False
SF  1        Kent Bazemore  False
    2      Thabo Sefolosha  False
PF  1         Paul Millsap  False
    2           Mike Scott  False
    3         Mike Muscala  False
C   1           Al Horford  False
    2       Tiago Splitter   True
    3       Walter Tavares  False
Brooklyn Nets
                             Name    Out
Pos Rank                                
PG  1                Jarrett Jack  False
    2                Shane Larkin  False
    3                Donald Sloan  False
SG  1             Wayne Ellington  False
    2            Bojan Bogdanovic  False
    3                Markel Brown  False
SF  1                 Joe Johnson  False
    2     Rondae Hollis-J